## Xây dựng model cho Conv Net:

![](../img/model_conv.png)

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

# Load data from MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Valitation set
X_val, y_val = X_train[50000:60000:], y_train[50000:60000:]
# Training set
X_train, y_train = X_train[:50000], y_train[:50000]
print(X_train.shape)
print(f"X train :\n {X_train[0:5,:]}")
print(f"y_train : {y_train[0:5]}")

(50000, 28, 28)
X train :
 [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
y_train : [5 0 4 1 9]


Dữ liệu input cho mô hình convolutional neural network là 1 tensor 4 chiều (N, W, H, D), trong bài này là ảnh xám nên W = H = 28, D = 1, N là số lượng ảnh cho mỗi lần training. Do dữ liệu ảnh ở trên có kích thước là (N, 28, 28) tức là (N, W, H) nên rần reshape lại thành kích thước N 28 28 * 1 để giống kích thước mà keras yêu cầu.

In [17]:
# Reshape data for keras
# n_samples training
def ReshapeInputX(list_x):
    return tuple([X.reshape(X.shape[0], 28, 28, 1) for X in list_x])

# 3. Reshape lại dữ liệu cho đúng kích thước mà keras yêu cầu
# X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
# X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)
# X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train, X_val, X_test = ReshapeInputX((X_train, X_val, X_test))
print(X_train.shape)



(50000, 28, 28, 1)


### One-hot encoded for label

In [18]:
def convertToOneHot(tupleY):
    return tuple([np_utils.to_categorical(Y, 10) for Y in tupleY])
Y_train, Y_val = convertToOneHot((y_train, y_val))
print(Y_train[0:10])


[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


### Define model

1. Model = Sequential() để nói cho keras là ta sẽ xếp các layer lên nhau để tạo model. Ví dụ input -> CONV -> POOL -> CONV -> POOL -> FLATTEN -> FC -> OUTPUT
2. Ở layer đầu tiên cần chỉ rõ input_shape của ảnh, input_shape = (W, H, D), ta dùng ảnh xám kích thước (28,28) nên input_shape = (28, 28, 1)
3. Khi thêm Convolutional Layer ta cần chỉ rõ các tham số: K (số lượng layer), kernel size (W, H), hàm activation sử dụng. cấu trúc: model.add(Conv2D(K, (W, H), activation='tên_hàm_activation'))
4. Khi thêm Maxpooling Layer cần chỉ rõ size của kernel, model.add(MaxPooling2D(pool_size=(W, H)))
5. Bước Flatten chuyển từ tensor sang vector chỉ cần thêm flatten layer.
6. Để thêm Fully Connected Layer (FC) cần chỉ rõ số lượng node trong layer và hàm activation sử dụng trong layer, cấu trúc: model.add(Dense(số_lượng_node activation='tên_hàm activation'))

In [19]:
model = Sequential()

# Thêm Convolutional layer với 32 kernel, kích thước kernel 3*3     
# dùng hàm sigmoid làm activation và chỉ rõ input_shape cho layer đầu tiên
model.add(Conv2D(32, (3, 3), activation='sigmoid', input_shape=(28,28,1)))

# Thêm Convolutional layer 2
model.add(Conv2D(32, (3, 3), activation='sigmoid'))

# Thêm Maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

# Flatten 
model.add(Flatten())

# Thêm fully connected layer vs 128 nodes dùng activate function là sigmoid
model.add(Dense(128, activation='sigmoid'))

# Thêm fully connected layer vs 10 nodes và dùng activate function là softmax
model.add(Dense(10, activation='softmax'))



### Compile model


In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 7. Thực hiện train model
H = model.fit(X_train, Y_train, validation_data=(X_val, Y_val),
              batch_size=32, epochs=10, verbose=1)

Epoch 1/10
1563/1563 [==============================] - 74s 47ms/step - loss: 0.2856 - accuracy: 0.9218 - val_loss: 0.0654 - val_accuracy: 0.9825
Epoch 2/10
1563/1563 [==============================] - 80s 51ms/step - loss: 0.0572 - accuracy: 0.9838 - val_loss: 0.0503 - val_accuracy: 0.9851
Epoch 3/10
1563/1563 [==============================] - 83s 53ms/step - loss: 0.0360 - accuracy: 0.9900 - val_loss: 0.0425 - val_accuracy: 0.9869
Epoch 4/10
1563/1563 [==============================] - 86s 55ms/step - loss: 0.0246 - accuracy: 0.9930 - val_loss: 0.0420 - val_accuracy: 0.9877
Epoch 5/10
1563/1563 [==============================] - 79s 50ms/step - loss: 0.0159 - accuracy: 0.9955 - val_loss: 0.0405 - val_accuracy: 0.9888
Epoch 6/10
1563/1563 [==============================] - 80s 51ms/step - loss: 0.0107 - accuracy: 0.9973 - val_loss: 0.0452 - val_accuracy: 0.9876
Epoch 7/10
1563/1563 [==============================] - 80s 51ms/step - loss: 0.0077 - accuracy: 0.9979 - val_loss: 0.0406 -